In [7]:
!pip install -r requirements.txt

In [28]:
import base64
import json

from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from cryptography.fernet import Fernet

### read in pub and private key

In [9]:
with open("keys/private_key.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(
        f.read(), password=None, backend=default_backend()
    )

with open("keys/public_key.pem", "rb") as f:
    public_key = serialization.load_pem_public_key(f.read(), backend=default_backend())

### read in response.json

In [10]:
with open("response/response.json", "r") as f:
    response = json.load(f)

In [11]:
response

{'emailEncrypted': 'gAAAAABlsljFJZYbk3pRiowWKk-DkU5wpAecGyNt_FvhsRStwP3aVfC35Ah1cWBjGJiWuUmE90TTw4Ho61_0A_nZ1lM1aU1PGVi0z1jhPai2hLmpU2frNhk=',
 'base64EncodedKey': ['sfD5nvAmzRZyWJu6gEEPZbdOBER+4O+ud8SEI5U7IgFqiLijkdiD+dFaCSWnQCnlRx/V1afpRa8E2yCrcBJj9hMKji7oDwMkWFih1q5iznrRqQ4quck+Z2W34vYrhtM3Lo0kvmHkEhm5z5BWZ0vyvl94TAoGuh0NdfEBYEk7btf+DipH07gMPAhiJfC10rEgNE7l+Ttz+43P70CDjB4QjNHd6FuC/xPr3rug4NBh9v2qLljb+LkIRSEGkvdaKdHCjCcymbEiaLvFYuoAcbDV3Hyj/z9XT2UTR1s5sipsg7gHZ2qHDKPuRac3abNJB2EUvSZchImfGcFf91XcPp/t4fzq9Xm6osTjHgXTGDTry+pfAIlb5OO/eMUtXoHFLt32Y7IfE7oyIS2r5+AC4nHf7KG0S6tnkDE48OdY4N7DHZ/uCxevLFxmKjwyF/eVmbqlg75dp8zoyn5dcGJ4lezNSVtL0zGFF33RM8SzXlZLkSdd22OAhEaFKaaAW4qyT7olIeEzPZz011XD+plkL5xTjNV/fOiof3ZjqcsLr29CqfEnapEyserm19/9UHAVtyXlNnyLQRKG5PMPWmd2flnVgCSadIHKgSHh9hRIG9rGDr1hyugegM6dua+OoaqD9aqx5yR9p1MNVp/x9kueYadxFL238+g6F+gsx6tOwLIBxc0=']}

### 1. convert base64 RSA to bytes

In [16]:
base64EncodedKey = response.get("base64EncodedKey", [None])[0]

In [19]:
EncodedKey = base64.b64decode(base64EncodedKey)

### 2. decrypt RSA public key to get symetric AES key

In [29]:
aes_key = private_key.decrypt(
    EncodedKey,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    ),
)

In [30]:
cipher = Fernet(aes_key)

### 3. use AES key to decrypt message

In [36]:
emailEncrypted = response.get("emailEncrypted", None)
decrypted_email = cipher.decrypt(emailEncrypted).decode('utf-8')
decrypted_email

'loremipsum@email.com'